In [1]:
pip install transformers

In [2]:
pip install torch

In [3]:
!pip install transformers accelerate peft

In [9]:
import pandas as pd

# dataset_path = '/content/NLP_ass_test.tsv'  # Replace with your dataset path
# data = pd.read_csv(dataset_path)
# posts = data['post_column_name']


test_df = pd.read_csv('/content/NLP_ass_test.tsv', sep='\t', header=None)

In [11]:
test_df.columns = ['statement', 'category']

In [12]:
test_df

,statement,category
0,i dont think im getting my baby them white 9 h...,normal
1,laura loomer raped me while screaming at me in...,hatespeech
2,<user> what did the old lady do was she a nazi...,normal
3,as much as i appreciate a format of open discu...,normal
4,sex be so good a bitch be slow stroking and cr...,offensive
...,...,...
1919,it always women trying this shit like dingbat ...,offensive
1920,because women would never lie about being sexu...,offensive
1921,how is all that awesome muslim diversity going...,offensive
1922,well my dear lgbtq brothers and sisters i do n...,hatespeech


In [16]:
model_name = "google/flan-t5-small"  # or "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# New Section

In [40]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

In [39]:
model = get_peft_model(model, lora_config)

In [42]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [60]:
from datasets import Dataset
dataset = Dataset.from_pandas(test_df)


# Define the tokenization function
def tokenize_fn(examples):
    inputs = tokenizer(examples["statement"], padding="max_length", truncation=True)
    outputs = tokenizer(examples["category"], padding="max_length", truncation=True)
    inputs['labels'] = outputs['input_ids']
    return inputs

# Split into train and validation datasets (e.g., 80-20 split)
train_data, val_data = dataset.train_test_split(test_size=0.2).values()

# Tokenize train and validation data
train_tokenized = train_data.map(tokenize_fn, batched=True)
val_tokenized = val_data.map(tokenize_fn, batched=True)


Map:   0%|          | 0/1539 [00:00<?, ? examples/s]

Map:   0%|          | 0/385 [00:00<?, ? examples/s]

In [62]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    run_name="7e0df72cb3a9a6c2655a4fbab1a0cf1ecb9fa6e9",  # Specify a unique run name here
)

In [63]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)

In [52]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [65]:
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

def tokenize_fn(examples):
    inputs = tokenizer(examples["statement"], padding="max_length", truncation=True, return_tensors="pt")
    outputs = tokenizer(examples["category"], padding="max_length", truncation=True, return_tensors="pt")
    inputs['labels'] = outputs['input_ids']
    return inputs

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [66]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [67]:
print(train_tokenized[0])

{'statement': 'suck out you faggot', 'category': 'offensive', 'input_ids': [3, 7, 4636, 91, 25, 3, 89, 9, 122, 10779, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [69]:
print(len(train_tokenized))
print(len(val_tokenized))

1539
385


In [70]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
)


In [71]:
print("Train dataset:", train_tokenized)
print("Validation dataset:", val_tokenized)

Train dataset: Dataset({
    features: ['statement', 'category', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1539
})
Validation dataset: Dataset({
    features: ['statement', 'category', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 385
})


In [74]:
trainer.save_model("fine_tuned_flan_t5")

In [82]:
import torch  # Ensure you have this import at the top

# Set the model to evaluation mode
model.eval()

# Iterate over the tokenized test data
for example in tokenized_dataset:
    # Prepare inputs for the model
    inputs = {}
    for key, val in example.items():
        if key != 'labels':
            # If val is already a tensor, just add it with an additional dimension
            if isinstance(val, torch.Tensor):
                inputs[key] = val.unsqueeze(0)  # Add batch dimension
            # Check if val is a list of input IDs
            elif isinstance(val, list):
                inputs[key] = torch.tensor(val).unsqueeze(0)  # Convert to tensor and add batch dimension
            else:
                # For any unexpected types, you can handle them or skip
                print(f"Warning: Unexpected type for {key}: {type(val)}")

    with torch.no_grad():
        outputs = model.generate(**inputs)

    # Decode the output and print
    print("Input:", example["statement"])
    print("Prediction:", tokenizer.decode(outputs[0], skip_special_tokens=True))
    print("Actual:", example["category"])  # Use the appropriate key for actual labels
    print("-" * 50)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Actual: normal
--------------------------------------------------
Input: <user> right here bro found jesus whos not just a prophet howz life u an ex muslim too from yeg
Prediction: user> right here bro found jesus whos not just a prophet how
Actual: normal
--------------------------------------------------
Input: these hoes really be talking bout another hoe and still be in that hoe face
Prediction: i think i'm gonna be in that hoe face
Actual: offensive
--------------------------------------------------
Input: who do bitches in horror movies always have huge houses like cmon we know you can ’ t cover those property taxes
Prediction: a slasher
Actual: offensive
--------------------------------------------------
Input: when i get married i want my wedding to be like the one off crazy rich asians
Prediction: i want my wedding to be like the one off crazy rich asians
Actual: normal
--------------------------------------------------
Input: